# 🏋️ AthenAI Exercise Database Refactor & Export

This notebook will:
1. Load Kaggle and GitHub exercise datasets.
2. Normalize all fields to match your domain schema (see public.go).
3. Deduplicate exercises and preserve all domain-relevant data.
4. Build DataFrames for each table and join table (exercise, muscular_group, equipment, exercise_muscular_group, exercise_equipment, etc.).
5. Export each as a CSV for bulk import into PostgreSQL.
6. Download CSVs for local import (no direct DB connection needed).

In [ ]:
# Install required libraries
%pip install pandas requests kaggle --quiet

## 🔑 Kaggle API Setup
Upload your kaggle.json file to access the Kaggle dataset.

In [ ]:
from google.colab import files # type: ignore
files.upload()  # Upload kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

## 📥 Load Datasets

In [ ]:
import pandas as pd # type: ignore
import requests # type: ignore

# Kaggle dataset
!kaggle datasets download -d niharika41298/gym-exercise-data -p /content --unzip
df_kaggle = pd.read_csv('/content/megaGymDataset.csv')

# GitHub dataset
url = 'https://raw.githubusercontent.com/yuhonas/free-exercise-db/main/dist/exercises.json'
df_github = pd.read_json(url)

## 🔄 Normalize to Domain Schema
Map all fields needed for your public.exercise, muscular_group, equipment, and join tables.

In [ ]:
def canonicalize(name):
    if not isinstance(name, str): return None
    name = name.strip().lower()
    norm_map = {
        'kettlebells': 'kettlebell', 'dumbbells': 'dumbbell', 'barbells': 'barbell',
        'bands': 'resistance band', 'band': 'resistance band',
        'body only': 'bodyweight', 'bodyweight': 'bodyweight',
        'machines': 'machine', 'machine': 'machine',
        'cable': 'cable', 'e-z curl bar': 'barbell',
        'exercise ball': 'exercise ball', 'medicine ball': 'medicine ball',
        'foam roll': 'foam roll', 'other': 'other', 'accessories': 'other'
    }
    return norm_map.get(name, name)

def normalize_kaggle(row):
    return {
        'name': row.get('Title'),
        'difficulty_level': (row.get('Level') or 'beginner').lower(),
        'exercise_type': (row.get('Type') or 'strength').lower(),
        'instructions': row.get('Desc'),
        'video_url': None,
        'image_url': None,
        'muscular_groups': [canonicalize(row.get('BodyPart'))] if row.get('BodyPart') else [],
        'equipment': [canonicalize(row.get('Equipment'))] if row.get('Equipment') else []
    }

def normalize_github(row):
    return {
        'name': row.get('name'),
        'difficulty_level': (row.get('level') or 'beginner').lower(),
        'exercise_type': (row.get('category') or 'strength').lower(),
        'instructions': ' '.join(row.get('instructions', [])) if isinstance(row.get('instructions'), list) else None,
        'video_url': row.get('videoURL'),
        'image_url': row['images'][0] if isinstance(row.get('images'), list) and len(row['images']) > 0 else None,
        'muscular_groups': [canonicalize(mg) for mg in row.get('primaryMuscles', []) + row.get('secondaryMuscles', []) if mg],
        'equipment': [canonicalize(row.get('equipment'))] if row.get('equipment') else []
    }

clean_kaggle = [normalize_kaggle(r) for r in df_kaggle.to_dict(orient='records')]
clean_github = [normalize_github(r) for r in df_github.to_dict(orient='records')]

df_clean = pd.DataFrame(clean_kaggle + clean_github)

## 🧹 Deduplicate Exercises
Deduplicate by name and preserve all domain-relevant data.

In [ ]:
df_clean['name_lower'] = df_clean['name'].str.lower()
df_clean = df_clean.drop_duplicates(subset=['name_lower']).drop(columns=['name_lower'])
print('Final exercise count:', len(df_clean))

## 🗂️ Build DataFrames for Each Table
Create DataFrames for exercise, muscular_group, equipment, and join tables.

In [ ]:
# Exercise table (no synonyms column)
exercise_cols = ['name', 'difficulty_level', 'exercise_type', 'instructions', 'video_url', 'image_url']
df_exercise = df_clean[exercise_cols].copy()
df_exercise.to_csv('exercise.csv', index=False)

# Muscular group table (from all unique canonical muscular groups)
all_mg = set(mg for ex in df_clean['muscular_groups'] for mg in ex if mg)
df_mg = pd.DataFrame({'name': sorted(list(all_mg))})
df_mg.to_csv('muscular_group.csv', index=False)

# Equipment table (from all unique canonical equipment)
all_eq = set(eq for ex in df_clean['equipment'] for eq in ex if eq)
df_eq = pd.DataFrame({'name': sorted(list(all_eq))})
df_eq.to_csv('equipment.csv', index=False)

# Join tables
exercise_mg_rows = []
for idx, ex in df_clean.iterrows():
    for mg in ex['muscular_groups']:
        exercise_mg_rows.append({'exercise_name': ex['name'], 'muscular_group_name': mg})
df_exercise_mg = pd.DataFrame(exercise_mg_rows)
df_exercise_mg.to_csv('exercise_muscular_group.csv', index=False)

exercise_eq_rows = []
for idx, ex in df_clean.iterrows():
    for eq in ex['equipment']:
        exercise_eq_rows.append({'exercise_name': ex['name'], 'equipment_name': eq})
df_exercise_eq = pd.DataFrame(exercise_eq_rows)
df_exercise_eq.to_csv('exercise_equipment.csv', index=False)

## 💾 Download CSVs for Local Import
Download all CSVs to your local machine for bulk import into PostgreSQL.

In [ ]:
from google.colab import files # type: ignore
for fname in ['exercise.csv', 'muscular_group.csv', 'equipment.csv', 'exercise_muscular_group.csv', 'exercise_equipment.csv']:
    files.download(fname)